# Sorting brain regions by DICE score

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%run nifti_tools.ipynb

In [2]:
# Loading the comprehensive overlap dataframe of brain regions and Jacobian
df_path = '/data/bioprotean/RAG2/AVG/MWT_avg/to_allen/overlap/200um/RAG2_neg_comp_overlap_anat.csv'
overlap_df = pd.read_csv(df_path, index_col = 'Unnamed: 0')

In [3]:
overlap_df.head()

,full_ant_index,structure,acronym,ish_id,ant_id,in_200_atlas,region_overlap_ratio,jac_overlap_ratio,DICE_neg_jcb_anat
0,0,"""root""","""root""",-1.0,997,True,0.069196,0.010389,0.018065
1,7,"""Frontal pole, layer 1""","""FRP1""",998.0,68,True,0.300000,0.002011,0.003995
2,8,"""Frontal pole, layer 2/3""","""FRP2/3""",1073.0,667,True,0.250000,0.002346,0.004648
3,19,"""Primary motor area, Layer 1""","""MOp1""",888.0,320,True,0.000000,0.000000,0.000000
4,20,"""Primary motor area, Layer 2/3""","""MOp2/3""",966.0,943,True,0.000000,0.000000,0.000000


In [4]:
def compute_DICE(region_ids_list, cluster_vec):
    '''
    Computing DICE score for brain regions and sorting them by that.
    
    Args:
        region_ids_list: list
            List of brain region IDs to visualize.
    
        cluster_vec: vec
            The path to the cluster file.
    
    Returns:
        DICE_list: list
            List of DICE scores
    '''
    
    # Loading the anatomy file
    ant_file = '/data/bioprotean/RAG2/AVG/MWT_avg/to_allen/overlap/200um/allen_annot200.nii'
    ant_vec = nifti_to_array(ant_file)
    
    # Making a list to store top DICE scores
    DICE_list = []
    
    # Getting number of clusters in a list
    n_clusters = np.unique(cluster_vec)
    cluster_ids = list(n_clusters)
    
    for ID in region_ids_list:
        # Selecting a brain region
        region_ID = [ID]
        
        # Computing the overlap for the cluster
        overlap_list = overlap_function(base_arr = ant_vec, input_arr = cluster_vec,\
        base_ids_list = region_ID, input_ids_list = cluster_ids)
        
        # Selecting the top overlap ratio
        top_DICE = max(overlap_list)
        DICE_list.append(top_DICE)
    
    return DICE_list

In [5]:
# Reconstructing the labels
labels_path = '/data/bioprotean/ABA/SFT/clusters/100features_594_clusters.npy'
labels_arr = np.load(labels_path)

indices_path = '/data/bioprotean/ABA/MEMMAP/genes_list/half_indices.npy'
labels = reconstruct_ABA(labels_arr, indices_path)

In [ ]:
# Computing the DICE for all brain regions

# Loading the region IDs list
region_ids_list = overlap_df['ant_id']

# Inputting to the function
DICE_list = compute_DICE(region_ids_list, labels)

In [16]:
# Adding the DICE scores to the dataframe
overlap_df['DICE'] = DICE_list

In [17]:
overlap_df.head()

,full_ant_index,structure,acronym,ish_id,ant_id,in_200_atlas,region_overlap_ratio,jac_overlap_ratio,DICE_neg_jcb_anat,DICE
0,0,"""root""","""root""",-1.0,997,True,0.069196,0.010389,0.018065,0.210396
1,7,"""Frontal pole, layer 1""","""FRP1""",998.0,68,True,0.300000,0.002011,0.003995,0.277778
2,8,"""Frontal pole, layer 2/3""","""FRP2/3""",1073.0,667,True,0.250000,0.002346,0.004648,0.320000
3,19,"""Primary motor area, Layer 1""","""MOp1""",888.0,320,True,0.000000,0.000000,0.000000,0.308824
4,20,"""Primary motor area, Layer 2/3""","""MOp2/3""",966.0,943,True,0.000000,0.000000,0.000000,0.223022


In [18]:
# Saving the dataframe to file
overlap_df.to_csv('/data/bioprotean/ABA/SFT/100features_df.csv')